# Real Data Examples

In this notebook we will fit our model on the two example datasets provided in the gcmr package:

    1. gcmr: Epilepsy 
    2. geepack: Respiratory

For these examples we will use the autoregressive AR(1) parameterization of the covariance matrix $\Gamma,$ estimating correlation parameter $\rho$ and dispersion parameter $\sigma^2$. 

    note: For the dispersion parameter, we can an L2 penalty to the loglikelihood to keep the estimates from going off to infinity. This notebook presents results with the unpenalized fit.

In [1]:
versioninfo()

Julia Version 1.6.2
Commit 1b93d53fc4 (2021-07-14 15:36 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.7.0)
  CPU: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, skylake)


In [ ]:
using CSV, DataFrames, GLMCopula, LinearAlgebra, GLM, RCall, RData, RDatasets

## Example 1: Poisson AR(1)

We first demonstrate how to fit the model with Poisson base, and AR(1) covariance on the "epilepsy" dataset from the "gcmr" package in R.

In [ ]:
R"""
    library("gcmr")
    data("epilepsy", package = "gcmr")
"""
@rget epilepsy;

Let's take a preview of the first 10 lines of the epilepsy dataset.

In [ ]:
epilepsy[1:10, :]

To form the model, we give it the following arguments:

- named dataframe
- outcome variable name of interest as a symbol
- grouping variable name of interest as a symbol
- covariate names of interest as a vector of symbols
- base distribution
- link function


In [ ]:
df = epilepsy
y = :counts
grouping = :id
covariates = [:visit, :trt]
d = Poisson()
link = LogLink()

Poisson_AR_model = AR_model(df, y, grouping, covariates, d, link);

Fit the model

In [ ]:
GLMCopula.fit!(Poisson_AR_model, IpoptSolver(print_level = 5, max_iter = 100, tol = 10^-8, limited_memory_max_history = 20, hessian_approximation = "limited-memory"));

We can take a look at the MLE's

In [ ]:
@show Poisson_AR_model.β
@show Poisson_AR_model.σ2
@show Poisson_AR_model.ρ;

Calculate the loglikelihood at the maximum

In [ ]:
@show loglikelihood!(Poisson_AR_model, true, true);

## Example 2: Bernoulli AR(1) 


We will next demo how to fit the model with Bernoulli base and AR(1) covariance on the "respiratory" dataset from the "geepack" package. 

In [ ]:
R"""
    data(respiratory, package="geepack")
    respiratory_df <- respiratory[order(respiratory$id),]
"""

@rget respiratory_df;

Let's take a preview of the first 10 lines of the respiratory dataset in long format.

In [ ]:
respiratory_df[1:10, :]

To form the model, we give it the following arguments:

- named dataframe
- outcome variable name of interest as a symbol
- grouping variable name of interest as a symbol
- covariate names of interest as a vector of symbols
- base distribution
- link function

In [ ]:
df = respiratory_df
y = :outcome
grouping = :id
covariates = [:center, :age, :baseline]
d = Bernoulli()
link = LogitLink()

Bernoulli_AR_model = AR_model(df, y, grouping, covariates, d, link);

Fit the model

In [ ]:
GLMCopula.fit!(Bernoulli_AR_model, IpoptSolver(print_level = 5, max_iter = 100, tol = 10^-8, limited_memory_max_history = 20, hessian_approximation = "limited-memory"));

We can take a look at the MLE's

In [ ]:
@show Bernoulli_AR_model.β
@show Bernoulli_AR_model.σ2
@show Bernoulli_AR_model.ρ;

Calculate the loglikelihood at the maximum

In [ ]:
@show loglikelihood!(Bernoulli_AR_model, true, true);